In [2]:
import os
from torch.utils.data import DataLoader
from dataloader_e2e import create_dataset_12
import matplotlib.pyplot as plt
from PIL import ImageOps
import torchvision
import numpy as np
import cv2

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import *


device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)
path = os.getcwd()
path = '/root/notebooks/sample/dataset/ECG100_224_new/'
train_dataloader,test_dataloader,label_lead, labels=create_dataset_12(path,1,None)



cpu
Classification lab : {'Lateral': 0, 'Posterior': 1, 'Anterior': 2, 'Inferior': 3, 'Normal': 4}
Number of datas 368
torch.Size([12, 368, 3, 112, 224])
368
368


In [ ]:
# モデルクラス
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # 畳み込み層や全結合層などを定義する
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.faltten = nn.Flatten()
        self.fc1 = nn.Linear(16,16)
        self.fc2 = nn.Linear(16, 12)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.gap(x)
        x = self.faltten(x)
        #x = x.view(-1, 16 * 56 * 112)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# モデルのインスタンスを作成
model = CNNModel().to(device)


事前学習

In [ ]:
# モデル訓練　事前学習12クラス分類

# 損失関数
criterion = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(model.parameters(), lr=0.0001)
bestscore = 1000.0
loss_list,loss_list_test = [], []
f1s, f1s_test = [],[]
acc, acc_test = [], []

num_epochs = 4000

for epoch in range(num_epochs):
    model.train()
    running_loss, running_loss_test = 0.0, 0.0
    preds, label =[], []
    preds_test,label_test = [], []

    for ecg_image,labels,label_lead,patients,num_leads in train_dataloader:
        image =ecg_image.reshape(-1,3,112,224).to(device)
        
        label_lead = label_lead.reshape(-1).to(device)
        optimizer.zero_grad()  # 勾配を初期化
        
        outputs = model(image)  # モデルの出力を計算

        prob, pred=torch.max(outputs,1)
        preds.extend(pred.tolist())
        label.extend(label_lead.tolist())
        loss = criterion(outputs, label_lead)  # 損失を計算
        loss.backward()  # 勾配を計算
        optimizer.step()  # パラメータを更新

        running_loss += loss.item()


    for ecg_image,labels,label_lead,patients,num_leads in test_dataloader:
        image =ecg_image.reshape(-1,3,112,224).to(device)

        label_lead = label_lead.reshape(-1).to(device)

        outputs = model(image)  # モデルの出力を計算
        
        prob, pred=torch.max(outputs,1)
        preds_test.extend(pred.tolist())
        label_test.extend(label_lead.tolist())
        
        loss = criterion(outputs, label_lead)  # 損失を計算

        running_loss_test += loss.item()
    
    score = running_loss_test/len(test_dataloader)
    if score < bestscore:
        bestscore = score
        torch.save(model.state_dict(), '/root/notebooks/sample/models/bestmodel.pth')
        print('best score!!')
    loss_list.append(running_loss/len(train_dataloader))
    loss_list_test.append(running_loss_test/len(test_dataloader))
    f1 = f1_score(preds,label,average='macro')
    f1_test = f1_score(preds_test,label_test,average='macro')
    f1s.append(f1)
    f1s_test.append(f1_test)
    accuracy = accuracy_score(preds,label)
    accuracy_test = accuracy_score(preds_test,label_test)
    acc.append(accuracy)
    acc_test.append(accuracy_test)

    # 1エポック終了時の損失を表示
    print(f"Epoch [{epoch+1}/{num_epochs}] Train-Loss: {running_loss/len(train_dataloader)} Test-Loss: {running_loss_test/len(train_dataloader)} Train-f1: {f1} Test-f1: {f1_test} Train-acc: {accuracy} Test-acc: {accuracy_test}")

In [ ]:
# モデル評価
pred,label = [],[]
model.load_state_dict(torch.load('/root/notebooks/sample/models/bestmodel.pth'))
model.eval()

for ecg_image,labels,label_lead,patients,num_leads in test_dataloader:
    image =ecg_image.reshape(-1,3,112,224).to(device)
    label_lead = label_lead.reshape(-1).to(device)
    label.extend(label_lead.tolist())
    outputs = model(image)  # モデルの出力を計算
    prob, preds=torch.max(outputs,1)
    pred.extend(preds.tolist())
    f1_test = f1_score(preds.tolist(),label_lead.tolist(),average='macro')
    
    loss = criterion(outputs, label_lead)  # 損失を計算

    running_loss_test += loss.item()

In [ ]:
# 学習曲線 (損失関数) 事前学習
plt.figure(figsize=(8,6))
plt.plot(loss_list,label='train', lw=3, c='b')
plt.plot(loss_list_test,label='test', lw=3, c='r')
plt.title('Loss')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

# 学習曲線 (accuracy)
plt.figure(figsize=(8,6))
plt.plot(acc,label='acc_train', lw=3, c='b')
plt.plot(acc_test,label='acc_test', lw=3, c='r')
plt.title('accuracy')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()


In [ ]:
# Confusion matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
           """
           This function prints and plots the confusion matrix.
           Normalization can be applied by setting `normalize=True`.
           """
           if normalize:
               cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
               print("Normalized confusion matrix")
           else:
               print('Confusion matrix, without normalization')
           print(cm)
           plt.imshow(cm, interpolation='nearest', cmap=cmap)
           plt.title(title)
           plt.colorbar()
           tick_marks = np.arange(len(classes))
           plt.xticks(tick_marks, classes, rotation=45)
           plt.yticks(tick_marks, classes)
           fmt = '.2f' if normalize else 'd'
           thresh = cm.max() / 2.
           for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
               plt.text(j, i, format(cm[i, j], fmt),
                        horizontalalignment="center",
                        color="white" if cm[i, j] > thresh else "black")
           plt.tight_layout()
           plt.ylabel('True label')
           plt.xlabel('Predicted label')
           #plt.savefig(output_file)


labels=[0,1,2,3,4,5,6,7,8,9,10,11]
label2=['Ⅰ','Ⅱ','Ⅲ','aVR','aVL','aVF','V1', 'V2', 'V3', 'V4', 'V5','V6']
cnf_matrix=confusion_matrix(label,pred,labels=labels)
title="confusion matrix"
plt.figure(figsize=(4,4),dpi=400)
plot_confusion_matrix(cnf_matrix, classes=label2,title=title)
plt.show()

ファインチューニング

In [ ]:
# CNNモデルの特徴抽出器を取り出す
feature_extractor = nn.Sequential(*list(model.children())[:-2])

# 新しい全結合層を追加
feature_extractor.to(device)
print(feature_extractor)


# カスタムレイヤーを定義
class MeanLayer(nn.Module):
    def forward(self, x):
        return torch.mean(x, dim=0, keepdim=True)

# nn.Sequential内にカスタムレイヤーを含むモデルを定義
CNN_diagnosis = nn.Sequential(
    feature_extractor,
    MeanLayer(),
    nn.Linear(16, 16),
    nn.Linear(16, 5)
)

CNN_diagnosis.to(device)

print(CNN_diagnosis)

input_shape = (12, 3, 112, 224)

# ランダムなテンソルを生成
random_input = torch.randn(input_shape).to(device)

# モデルにランダムなテンソルを渡して出力を得る
random_output = CNN_diagnosis(random_input).to(torch.float64)
feature_output = feature_extractor(random_input).to(torch.float64)
print(random_output.shape)
print(feature_output.shape)

In [ ]:
## 診断モデル

# 損失関数
criterion = nn.CrossEntropyLoss()


# 最適化アルゴリズム
optimizer = optim.Adam(CNN_diagnosis.parameters(), lr=0.0001)
bestscore_dx = 1000.0
loss_list_dx,loss_list_test_dx = [], []
f1s_dx, f1s_test_dx = [],[]
acc_dx, acc_test_dx = [], []

num_epochs_dx = 2000

# モデル訓練　5クラス分類

for epoch in range(num_epochs_dx):
    CNN_diagnosis.train()
    running_loss_dx = 0.0
    preds_dx, label_dx = [], []
    running_loss_dx, running_loss_test_dx = 0.0, 0.0
    preds_test_dx,label_test_dx = [], []

    for ecg_image, labels, label_lead, patients, num_leads in train_dataloader:
        image = ecg_image.reshape(-1, 3, 112, 224).to(device)
        labels = labels.reshape(-1).to(device)  # ラベルを1次元に変形
        labels = labels.to(torch.long)  # CrossEntropyLossではクラスのインデックスはlong型である必要がある

        optimizer.zero_grad()  # 勾配を初期化

        outputs = CNN_diagnosis(image).to(torch.float64)  # モデルの出力を計算

        prob, pred = torch.max(outputs, 1)
        preds_dx.extend(pred.tolist())
        label_dx.extend(labels.tolist())

        loss = criterion(outputs, labels)  # 損失を計算（CrossEntropyLossの引数は予測とラベル）
        loss.backward()  # 勾配を計算
        optimizer.step()  # パラメータを更新

        running_loss_dx += loss.item()

    for ecg_image,labels,label_lead,patients,num_leads in test_dataloader:
        image =ecg_image.reshape(-1,3,112,224).to(device)
        labels = labels.reshape(-1).to(device)

        outputs = CNN_diagnosis(image)  # モデルの出力を計算
        outputs = outputs.to(torch.float64)
        prob, pred=torch.max(outputs,1)
        preds_test_dx.extend(pred.tolist())
        label_test_dx.extend(labels.tolist())
        
        #print(outputs)
        loss = criterion(outputs, labels)  # 損失を計算

        running_loss_test_dx += loss.item()
    
    score_dx = running_loss_test_dx/len(test_dataloader)
    if score_dx < bestscore_dx:
        bestscore_dx = score_dx
        torch.save(CNN_diagnosis.state_dict(), '/root/notebooks/sample/models/bestmodel_dx.pth')
        print('best score_dx!!')
    loss_list_dx.append(running_loss_dx/len(train_dataloader))
    loss_list_test_dx.append(running_loss_test_dx/len(test_dataloader))
    f1_dx = f1_score(preds_dx,label_dx,average='macro')
    f1_test_dx = f1_score(preds_test_dx,label_test_dx,average='macro')
    f1s_dx.append(f1_dx)
    f1s_test_dx.append(f1_test_dx)
    accuracy_dx = accuracy_score(preds_dx,label_dx)
    accuracy_test_dx = accuracy_score(preds_test_dx,label_test_dx)
    acc_dx.append(accuracy_dx)
    acc_test_dx.append(accuracy_test_dx)

    # 1エポック終了時の損失を表示
    print(f"Epoch [{epoch+1}/{num_epochs_dx}] Train-Loss: {running_loss_dx/len(train_dataloader)} Test-Loss: {running_loss_test_dx/len(train_dataloader)} Train-f1: {f1_dx} Test-f1: {f1_test_dx} Train-acc: {accuracy_dx} Test-acc: {accuracy_test_dx}")

In [ ]:
# モデル評価 ５クラス診断
pred_dx,label_dx = [],[]
CNN_diagnosis.load_state_dict(torch.load('/root/notebooks/sample/models/bestmodel_dx.pth'))
CNN_diagnosis.eval()

for ecg_image,labels,label_lead,patients,num_leads in test_dataloader:
    image =ecg_image.reshape(-1,3,112,224).to(device)
    labels = labels.reshape(-1).to(device)
    label_dx.extend(labels.tolist())
    outputs = CNN_diagnosis(image)  # モデルの出力を計算
    prob_dx, preds_dx=torch.max(outputs,1)
    pred_dx.extend(preds_dx.tolist())
    f1_test_dx = f1_score(preds_dx.tolist(),labels.tolist(),average='macro')
    accuracy_test_dx = accuracy_score(preds_test_dx,label_test_dx)
    loss = criterion(outputs, labels)  # 損失を計算

    running_loss_test_dx += loss.item()


In [ ]:
# 学習曲線 (損失関数) 診断モデル
plt.figure(figsize=(8,6))
plt.plot(loss_list_dx,label='train_dx', lw=3, c='b')
plt.plot(loss_list_test_dx,label='test_dx', lw=3, c='r')
plt.title('Loss_dx')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

# 学習曲線 (accuracy)
plt.figure(figsize=(8,6))
plt.plot(acc_dx,label='acc_train_dx', lw=3, c='b')
plt.plot(acc_test_dx,label='acc_test_dx', lw=3, c='r')
plt.title('accuracy_dx')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

# 学習曲線 (f1)
plt.figure(figsize=(8,6))
plt.plot(f1s_dx,label='f1_train_dx', lw=3, c='b')
plt.plot(f1s_test_dx,label='f1_test_dx', lw=3, c='r')
plt.title('f1_dx')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

In [ ]:
labels_dx=[0,1,2,3,4]
label2_dx=['Lateral','Posterior', 'Anterior', 'Inferior', 'Normal']
cnf_matrix=confusion_matrix(label_dx,pred_dx,labels=labels_dx)
title="confusion matrix"
plt.figure(figsize=(4,4),dpi=400)
plot_confusion_matrix(cnf_matrix, classes=label2_dx,title=title)
plt.show()

In [ ]:
# モデルのインスタンスを作成
model_np = CNNModel().to(device)

model_np = nn.Sequential(*list(model_np.children())[:-2])

model_np = nn.Sequential(
    model_np,
    MeanLayer(),
    nn.Linear(16, 16),
    nn.Linear(16, 5)
)

model_np.to(device)




In [ ]:
# 診断 (事前学習なし)

# 損失関数
criterion = nn.CrossEntropyLoss()


# 最適化アルゴリズム
optimizer = optim.Adam(model_np.parameters(), lr=0.0001)
bestscore_np = 1000.0
loss_list_np,loss_list_test_np = [], []
f1s_np, f1s_test_np = [],[]
acc_np, acc_test_np = [], []

num_epochs_np = 7000

# モデル訓練　5クラス分類

for epoch in range(num_epochs_np):
    model_np.train()
    running_loss_np = 0.0
    preds_np, label_np = [], []
    running_loss_np, running_loss_test_np = 0.0, 0.0
    preds_test_np,label_test_np = [], []

    for ecg_image, labels, label_lead, patients, num_leads in train_dataloader:
        image = ecg_image.reshape(-1, 3, 112, 224).to(device)
        labels = labels.reshape(-1).to(device)  # ラベルを1次元に変形
        labels = labels.to(torch.long)  # CrossEntropyLossではクラスのインデックスはlong型である必要がある

        optimizer.zero_grad()  # 勾配を初期化

        outputs = model_np(image).to(torch.float64)  # モデルの出力を計算

        prob, pred = torch.max(outputs, 1)
        preds_np.extend(pred.tolist())
        label_np.extend(labels.tolist())

        loss = criterion(outputs, labels)  # 損失を計算（CrossEntropyLossの引数は予測とラベル）
        loss.backward()  # 勾配を計算
        optimizer.step()  # パラメータを更新

        running_loss_np += loss.item()

    for ecg_image,labels,label_lead,patients,num_leads in test_dataloader:
        image =ecg_image.reshape(-1,3,112,224).to(device)
        labels = labels.reshape(-1).to(device)

        outputs = model_np(image)  # モデルの出力を計算
        outputs = outputs.to(torch.float64)
        prob, pred=torch.max(outputs,1)
        preds_test_np.extend(pred.tolist())
        label_test_np.extend(labels.tolist())
        
        #print(outputs)
        loss = criterion(outputs, labels)  # 損失を計算

        running_loss_test_np += loss.item()
    
    score_np = running_loss_test_np/len(test_dataloader)
    if score_np < bestscore_np:
        bestscore_np = score_np
        torch.save(model_np.state_dict(), '/root/notebooks/sample/models/bestmodel_np.pth')
        print('best score_np!!')
    loss_list_np.append(running_loss_np/len(train_dataloader))
    loss_list_test_np.append(running_loss_test_np/len(test_dataloader))
    f1_np = f1_score(preds_np,label_np,average='macro')
    f1_test_np = f1_score(preds_test_np,label_test_np,average='macro')
    f1s_np.append(f1_np)
    f1s_test_np.append(f1_test_np)
    accuracy_np = accuracy_score(preds_np,label_np)
    accuracy_test_np = accuracy_score(preds_test_np,label_test_np)
    acc_np.append(accuracy_np)
    acc_test_np.append(accuracy_test_np)

    # 1エポック終了時の損失を表示
    print(f"Epoch [{epoch+1}/{num_epochs_np}] Train-Loss: {running_loss_np/len(train_dataloader)} Test-Loss: {running_loss_test_np/len(train_dataloader)} Train-f1: {f1_np} Test-f1: {f1_test_np} Train-acc: {accuracy_np} Test-acc: {accuracy_test_np}")

In [ ]:
# モデル評価 ５クラス診断(事前学習なし)
pred_np,label_np = [],[]
model_np.load_state_dict(torch.load('/root/notebooks/sample/models/bestmodel_np.pth'))
model_np.eval()

for ecg_image,labels,label_lead,patients,num_leads in test_dataloader:
    image =ecg_image.reshape(-1,3,112,224).to(device)
    labels = labels.reshape(-1).to(device)
    label_np.extend(labels.tolist())
    outputs = model_np(image)  # モデルの出力を計算
    prob_np, preds_np=torch.max(outputs,1)
    pred_np.extend(preds_np.tolist())
    f1_test_np = f1_score(preds_np.tolist(),labels.tolist(),average='macro')
    accuracy_test_np = accuracy_score(preds_test_np,label_test_np)
    loss = criterion(outputs, labels)  # 損失を計算

    running_loss_test_np += loss.item()


In [ ]:
# 学習曲線 (損失関数) 事前学習
plt.figure(figsize=(8,6))
plt.plot(loss_list_np,label='train_np', lw=3, c='b')
plt.plot(loss_list_test_np,label='test_np', lw=3, c='r')
plt.title('Loss_np')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

# 学習曲線 (accuracy)
plt.figure(figsize=(8,6))
plt.plot(acc_np,label='acc_train_np', lw=3, c='b')
plt.plot(acc_test_np,label='acc_test_np', lw=3, c='r')
plt.title('accuracy_np')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

# 学習曲線 (F1)
plt.figure(figsize=(8,6))
plt.plot(f1s_np,label='f1_train_np', lw=3, c='b')
plt.plot(f1s_test_np,label='f1_test_np', lw=3, c='r')
plt.title('f1_np')
plt.xticks(size=14)
plt.yticks(size=14)
plt.grid(lw=2)
plt.legend(fontsize=14)
plt.show()

In [ ]:
labels_np=[0,1,2,3,4]
label2_np=['Lateral','Posterior', 'Anterior', 'Inferior', 'Normal']
cnf_matrix=confusion_matrix(label_np,pred_np,labels=labels_np)
title="confusion matrix"
plt.figure(figsize=(4,4),dpi=400)
plot_confusion_matrix(cnf_matrix, classes=label2_np,title=title)
plt.show()